In [1]:
import os

# Change the current directory to root directory
new_directory = "../"
os.chdir(new_directory)

# Verify the current directory has changed
updated_directory = os.getcwd()
print("Updated Directory:", updated_directory)


Updated Directory: /media/toma/2TB_30May2023/OBJECT_DETECTION_L/object_detection


In [ ]:
import sys
sys.path.append('./')

from transformers import AutoImageProcessor, MaskFormerFeatureExtractor, MaskFormerForInstanceSegmentation
from PIL import Image, ImageDraw
import numpy as np
import requests
import torch

: 

In [ ]:
import os
os.getcwd()

: 

In [ ]:
import numpy as np

# Define two NumPy arrays
arr1 = np.array([1, 2, 3])
arr2 = np.array([4, 5, 6])

# Multiply the arrays element-wise
result = arr1 * arr2

# Print the result
print(result)

result = result * 3 + result * 255 * 25554446666
print(result)

: 

### reproducing error

In [ ]:
### reproducing error


from PIL import Image, ImageDraw
import numpy as np

mask_path = "data/coco_datasets/panoptic_val2017/000000039769.png"

mask = Image.open(mask_path)

# Convert the mask image to a NumPy array
mask_array = np.array(mask, dtype = np.int64)

# Extract the red, green, and blue channels from the array
red_channel = mask_array[:, :, 0]
green_channel = mask_array[:, :, 1]
blue_channel = mask_array[:, :, 2]

print(red_channel)
print(green_channel)
print(blue_channel)

# Calculate the value array using the provided formula
mask_array = red_channel + green_channel * 256 + blue_channel * 256 * 256

print(mask_array)


all_labels = np.unique(mask_array)

print(all_labels)

# expected output: [0, 8222595, 8225432, 8798150, 14466198, 12821912, 10898909]


: 

### main code

In [ ]:
image_path = "data/coco_datasets/val2017/000000039769.jpg"
mask_path = "data/coco_datasets/panoptic_val2017/000000039769.png"
image = Image.open(image_path)
mask = Image.open(mask_path)


# load MaskFormer fine-tuned on COCO panoptic segmentation
feature_extractor = MaskFormerFeatureExtractor.from_pretrained("facebook/maskformer-swin-tiny-coco")
model = MaskFormerForInstanceSegmentation.from_pretrained("facebook/maskformer-swin-tiny-coco")


# Convert the mask image to a NumPy array
image = np.array(image)
image = torch.from_numpy(image).permute(2, 0, 1)

print(image.shape)


mask_array = np.array(mask, dtype = np.int64)

# Extract the red, green, and blue channels from the array
red_channel = mask_array[:, :, 0]
green_channel = mask_array[:, :, 1]
blue_channel = mask_array[:, :, 2]

# Calculate the value array using the provided formula
mask_array = red_channel + green_channel * 256 + blue_channel * 256 * 256


all_labels = np.unique(mask_array)

print(all_labels)


parsed_dict_from_json = {
    "segments_info": [
        {"id": 0, "category_id": 0, "iscrowd": 0, "bbox": [0, 0, 640, 480], "area": 2750},
        {"id": 8222595, "category_id": 1, "iscrowd": 0, "bbox": [18, 54, 301, 415], "area": 53306}, 
        {"id": 8225432, "category_id": 2, "iscrowd": 0, "bbox": [349, 26, 291, 343], "area": 59627}, 
        {"id": 8798150, "category_id": 4, "iscrowd": 0, "bbox": [1, 0, 639, 474], "area": 174579}, 
        {"id": 14466198, "category_id": 7, "iscrowd": 0, "bbox": [42, 74, 133, 45], "area": 4068}, 
        {"id": 12821912, "category_id": 89, "iscrowd": 0, "bbox": [333, 80, 38, 106], "area": 2118}, 
        {"id": 10898909, "category_id": 58, "iscrowd": 0, "bbox": [0, 0, 640, 480], "area": 2750}
        ], 
    "file_name": "000000039769.png", 
    "image_id": 39769}


instance_id_to_semantic_id = {d["id"] : d["category_id"] for d in parsed_dict_from_json["segments_info"]}
print(instance_id_to_semantic_id)


inputs_pre = feature_extractor(image)
print(inputs_pre)

inputs = feature_extractor.encode_inputs(
                pixel_values_list=inputs_pre["pixel_values"], 
                segmentation_maps=[mask_array],
                instance_id_to_semantic_id=[instance_id_to_semantic_id],
                return_tensors="pt")


print(inputs["pixel_values"].shape)
print(inputs["pixel_mask"].shape)
print(len(inputs["mask_labels"]))
print(inputs["mask_labels"][0].shape)
print(len(inputs["class_labels"]))
print(inputs["class_labels"][0].shape)


outputs = model(**inputs)


print(outputs)
# print(outputs.loss)
# print(outputs.loss)


: 